# [FLAC (Free Lossless Audio Codec)](https://xiph.org/flac/)

* Lossless audio compression mostly is realized by
using a combination of linear prediction or a
transformation followed by entropy coding. The
linear predictor attempts to minimize the variance of
the difference signal between the predicted sample
value and its actual value. The entropy coder
(Huffmann, LZW) allocates short codewords to
samples with high probability of occurance and
longer codewords to samples with lower probability
and in this way reduces the average bit consumption. 

* In the context of signal encoding, entropy coding refeers to assign short codewords to hightly probable (quantization levels) levels and longer code-words to less probable levels, yilding a short average code-word length.

* FLAC was [started in 2000](https://en.wikipedia.org/wiki/FLAC#History). In 2003 it becomes as a project in the Xiph.Org Foundation.
* Lossless. Typically compress 2:1.
* [Supports](https://en.wikipedia.org/wiki/FLAC#Source_encoder) only fixed-point samples, not floating-point. It can handle any PCM bit resolution from 4 to 24 bits per sample, any sampling rate from 1 Hz to 65,535 Hz in 1 Hz increments or from 10 Hz to 655,350 Hz in 10 Hz increments, and any number of channels from 1 to 8.

## Encoding algorithm:
1. Split the input in blocks of samples (4096 by default).
2. Perform inter-channel decorrelation: mid = (left + right) / 2, side = left - right.
1. Intra-channel decorrelation: Decorrelate PCM samples using linear prediction. Available predictors are: Zero, Verbatim, Fixed Linear (fitting a simple polynomial to the signal) and FIR Linear (linear predictive coding (LPC)) ... pero vamos, que ni idea.
2. Encode the residuals using Golomb Coding and RLE (silences). Rice coding involves finding a single parameter that matches a signal's distribution, then using that parameter to generate the codes. As the distribution changes, the optimal parameter changes, so FLAC supports a method that allows the parameter to change as needed. The residual can be broken into several contexts or partitions, each with it's own Rice parameter.

In [ ]:
Lossless audio compression:

framming -> (inter-channel dec) -> Intra-channel decorrelation -> Entropy coding

In [ ]:
Framming: for *editibility* and limit the propagation of errors.


In [ ]:
https://xiph.org/flac/format.html#interchannel

Prediction

FLAC uses four methods for modeling the input signal:

    Verbatim. This is essentially a zero-order predictor of the signal. The predicted signal is zero, meaning the residual is the signal itself, and the compression is zero. This is the baseline against which the other predictors are measured. If you feed random data to the encoder, the verbatim predictor will probably be used for every subblock. Since the raw signal is not actually passed through the residual coding stage (it is added to the stream 'verbatim'), the encoding results will not be the same as a zero-order linear predictor.
    Constant. This predictor is used whenever the subblock is pure DC ("digital silence"), i.e. a constant value throughout. The signal is run-length encoded and added to the stream.
    Fixed linear predictor. FLAC uses a class of computationally-efficient fixed linear predictors (for a good description, see audiopak and shorten). FLAC adds a fourth-order predictor to the zero-to-third-order predictors used by Shorten. Since the predictors are fixed, the predictor order is the only parameter that needs to be stored in the compressed stream. The error signal is then passed to the residual coder.
    FIR Linear prediction. For more accurate modeling (at a cost of slower encoding), FLAC supports up to 32nd order FIR linear prediction (again, for information on linear prediction, see audiopak and shorten). The reference encoder uses the Levinson-Durbin method for calculating the LPC coefficients from the autocorrelation coefficients, and the coefficients are quantized before computing the residual. Whereas encoders such as Shorten used a fixed quantization for the entire input, FLAC allows the quantized coefficient precision to vary from subframe to subframe. The FLAC reference encoder estimates the optimal precision to use based on the block size and dynamic range of the original signal.


In [ ]:
Residual Coding

FLAC currently defines two similar methods for the coding of the error signal from the prediction stage. The error signal is coded using Rice codes in one of two ways: 1) the encoder estimates a single Rice parameter based on the variance of the residual and Rice codes the entire residual using this parameter; 2) the residual is partitioned into several equal-length regions of contiguous samples, and each region is coded with its own Rice parameter based on the region's mean. (Note that the first method is a special case of the second method with one partition, except the Rice parameter is based on the residual variance instead of the mean.)

The FLAC format has reserved space for other coding methods. Some possiblities for volunteers would be to explore better context-modeling of the Rice parameter, or Huffman coding. See LOCO-I and pucrunch for descriptions of several universal codes.

The samples in the prediction residual are assumed to be uncorrelated and therefore, may be coded indepently.

The problem of residual coding is therefore to find an appropiate form for the probability density function (PDF) of the distribution of residuals vaules so that they can be efficiently modelled.

The optimal number of low order bits to be transmitted directly is linearly related the the variance of the signal.

The Laplacian is defined as:

\begin{equation}
  p(x) = 1/sqrt{2}\sigma e^{\sqrt{ }/\sigma|x|}
\end{equation}

\begin{equation}
  E(|x|) = \sigma/\sqrt{2}
\end{equation}

In [ ]:
LPC Linear Predictive Coding

\begin{equation}
  \hat{s}(t) = \sum_{i=1}^p a_i s(t-i)
\end{equation}

$p$ the order of the predictor.

\begin{equation}
  e(t) = s(t) \hat{s}(t)
\end{equation}

Durbin's algorithm is used for computing LPC coefficients $a_i$.

Autocorrelation coefficients.

Approx 50% compression is achieved using 0-th prediction and the best prediction order provides 58%. Hence, for lossless compression it is important not to waste too much compute on the predictor an to perform the residual coding efficiently.

In most applications, a FIR predictor is used, and the coefficients
of the prediction filter $\hat{A(z)}$ are determined to 
minimize the mean-squared prediction error. Without the
quantizer and $\hat{B}(z)=0$, the predictor coefficients can
be found for the FIR case by simply solving a set of
linear equations.

A simple choice of a predictor from a small library of fixed
predictors can be also possible.

In general, a minimum mean-squared predictor for audio signals
will involve fractional coefficients, which must be quantized
and coded as part of the lossless representation, and the 
predictor must be implementes with fixed-point arithmetic.


In [ ]:
Intra-channel decorrelation

To remove redundancy by decorrelation the samples within a frame.

Most algorithms remove redundancy by LPC. Each output frame contains the LPC coefficients and the prediction errors.

$e[n]$ should be uncorrelated and therefore, have a flat frequency spectrum.

## Performance (TO-DO)
Download the WAV file [Arnesen: MAGNIFICAT 4. Et misericordia](http://www.lindberg.no/hires/test/2L-106_04_stereo_44k_16b_01.flac) and expand it. Compute the compression ratio. Compare with [gzip](http://www.gzip.org/).